# Level 2: Intermediate - LCEL Deep Dive & LangGraph Introduction

This notebook takes you deeper into LangChain Expression Language (LCEL) and introduces LangGraph for building stateful agent workflows.

## Learning Objectives
- Master LCEL: RunnablePassthrough, RunnableParallel, RunnableLambda, branching
- Understand LangGraph fundamentals: StateGraph, nodes, edges
- Build your first graph-based agent from scratch
- Work with conditional edges and cycles
- Learn state management in graphs

## Prerequisites
- Completed Notebook 01 (Entry Level)

---

**References:**
- [LangGraph Quickstart](https://docs.langchain.com/oss/python/langgraph/quickstart)
- [LangGraph: Build Stateful AI Agents](https://realpython.com/langgraph-python)
- [Graph API Overview](https://langchain-ai.github.io/langgraph/how-tos/)

## 1. Setup

In [1]:
# Import required libraries
import os
import sys
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Add parent directory to path for shared config
sys.path.append('..')

# Import global model configuration
from config import (
    GPT_MODEL, GEMINI_MODEL,
    GPT_MODEL_NAME, GEMINI_MODEL_NAME,
    get_model, list_available_models,
)

print(f"Using GPT model:    {GPT_MODEL_NAME}")
print(f"Using Gemini model: {GEMINI_MODEL_NAME}")
print()
list_available_models()

OpenAI client initialized  -> model: gpt-4o-mini
Google client initialized  -> model: gemini-3-flash-preview
Using GPT model:    gpt-4o-mini
Using Gemini model: gemini-3-flash-preview

Available Models:
-------------------------------------------------------
  gpt-4o-mini          -> ChatOpenAI(gpt-4o-mini)
  gemini-3-flash-preview -> ChatGoogleGenerativeAI(gemini-3-flash-preview)
-------------------------------------------------------


## 2. LCEL Advanced Patterns

LangChain Expression Language (LCEL) provides composable primitives for building complex chains.

### Key Runnables
| Runnable | Purpose |
|----------|---------|
| `RunnablePassthrough` | Pass input through unchanged (or add fields) |
| `RunnableParallel` | Run multiple chains in parallel |
| `RunnableLambda` | Wrap any Python function as a runnable |
| `RunnableBranch` | Conditional routing based on input |

In [2]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# RunnableParallel: run multiple chains simultaneously
prompt_summary = ChatPromptTemplate.from_messages([
    ("system", "Summarize the following topic in 1 sentence."),
    ("human", "{topic}"),
])

prompt_keywords = ChatPromptTemplate.from_messages([
    ("system", "List 5 keywords for the following topic, comma-separated."),
    ("human", "{topic}"),
])

# Build parallel chains
parallel_chain = RunnableParallel(
    summary=prompt_summary | GPT_MODEL | StrOutputParser(),
    keywords=prompt_keywords | GEMINI_MODEL | StrOutputParser(),
)

result = parallel_chain.invoke({"topic": "Quantum Computing"})
print("Summary (GPT):", result["summary"])
print("\nKeywords (Gemini):", result["keywords"])

Summary (GPT): Quantum computing is an advanced computational paradigm that leverages the principles of quantum mechanics to process information in fundamentally different ways than classical computers, potentially solving complex problems more efficiently.

Keywords (Gemini): Qubit, Superposition, Entanglement, Quantum Supremacy, Quantum Algorithms


In [3]:
# RunnableLambda: wrap any function as a chain step
def word_count(text: str) -> dict:
    """Count words and return enriched result."""
    words = text.split()
    return {"text": text, "word_count": len(words), "char_count": len(text)}

analyze = (
    ChatPromptTemplate.from_messages([
        ("system", "Explain the concept briefly."),
        ("human", "{concept}"),
    ])
    | GPT_MODEL
    | StrOutputParser()
    | RunnableLambda(word_count)
)

result = analyze.invoke({"concept": "neural networks"})
print(f"Explanation: {result['text'][:100]}...")
print(f"Word count: {result['word_count']}")
print(f"Character count: {result['char_count']}")

Explanation: Neural networks are a subset of machine learning models inspired by the structure and function of th...
Word count: 84
Character count: 561


In [4]:
# RunnablePassthrough with assign: add computed fields to the input
from langchain_core.runnables import RunnablePassthrough

enrich_chain = (
    RunnablePassthrough.assign(
        upper_topic=lambda x: x["topic"].upper(),
        topic_length=lambda x: len(x["topic"]),
    )
)

result = enrich_chain.invoke({"topic": "artificial intelligence"})
print("Enriched input:", result)

Enriched input: {'topic': 'artificial intelligence', 'upper_topic': 'ARTIFICIAL INTELLIGENCE', 'topic_length': 23}


## 3. Introduction to LangGraph

LangGraph lets you build **stateful, graph-based** agent workflows. Instead of linear chains, you define:

| Concept | Description |
|---------|-------------|
| **State** | A TypedDict that flows through the graph |
| **Nodes** | Functions that transform the state |
| **Edges** | Connections between nodes (can be conditional) |
| **START / END** | Special nodes marking graph entry/exit |

### Why LangGraph over plain LCEL?
- Supports **cycles** (agents that loop until done)
- Built-in **state management**
- **Persistence** for multi-turn conversations
- **Human-in-the-loop** capabilities

In [5]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Step 1: Define the State
class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

# Step 2: Define nodes (functions that process state)
def chatbot(state: ChatState):
    """The chatbot node - calls the LLM with current messages."""
    return {"messages": [GPT_MODEL.invoke(state["messages"])]}

# Step 3: Build the graph
graph_builder = StateGraph(ChatState)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Step 4: Compile
simple_graph = graph_builder.compile()

# Step 5: Invoke
result = simple_graph.invoke({"messages": [{"role": "user", "content": "What is LangGraph?"}]})
print("Final response:", result["messages"][-1].content)

Final response: As of my last knowledge update in October 2023, LangGraph is not a widely recognized term or concept in mainstream technology or academia. However, it could refer to a specific project, tool, or framework related to language processing, graph theory, or a combination of both. 

If LangGraph is a recent development or a niche project that emerged after my last update, I would recommend checking the latest resources, official documentation, or community discussions for the most accurate and up-to-date information. If you have more context or details about LangGraph, I would be happy to help clarify or provide insights based on that information!


## 4. Graph with Tools and Conditional Edges

Now let's build a more realistic graph where the agent can **decide** whether to call tools or respond directly. This creates a **cycle**: the agent loops until it has enough information.

In [6]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
import json

# Define tools
@tool
def search_knowledge(query: str) -> str:
    """Search an internal knowledge base for information."""
    knowledge = {
        "langchain": "LangChain is a framework for building LLM-powered applications with chains, agents, and memory.",
        "langgraph": "LangGraph extends LangChain with graph-based state machines for complex agent workflows.",
        "rag": "RAG combines retrieval with generation to ground LLM responses in factual data.",
        "lcel": "LCEL (LangChain Expression Language) uses the pipe operator to compose chain components.",
    }
    for key, value in knowledge.items():
        if key in query.lower():
            return value
    return f"No information found for: {query}"

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression safely."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

tools = [search_knowledge, calculate]
tool_map = {t.name: t for t in tools}

# Bind tools to model
model_with_tools = GPT_MODEL.bind_tools(tools)

In [7]:
# Build a full agent graph with conditional routing

def agent_node(state: ChatState):
    """Call the LLM with tools bound."""
    return {"messages": [model_with_tools.invoke(state["messages"])]}

def tool_node(state: ChatState):
    """Execute all pending tool calls."""
    outputs = []
    last_message = state["messages"][-1]
    for tool_call in last_message.tool_calls:
        tool_result = tool_map[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call["id"],
                name=tool_call["name"],
            )
        )
    return {"messages": outputs}

def should_continue(state: ChatState) -> str:
    """Decide whether to call tools or end."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build the graph
agent_graph = StateGraph(ChatState)
agent_graph.add_node("agent", agent_node)
agent_graph.add_node("tools", tool_node)

agent_graph.add_edge(START, "agent")
agent_graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
agent_graph.add_edge("tools", "agent")  # Loop back after tool execution

# Compile
compiled_agent = agent_graph.compile()

# Test: this will cause the agent to search, then respond
result = compiled_agent.invoke({
    "messages": [{"role": "user", "content": "What is LCEL? Also calculate 42 * 17."}]
})

print("Agent conversation:")
for msg in result["messages"]:
    role = msg.type if hasattr(msg, "type") else "unknown"
    content = msg.content if hasattr(msg, "content") else str(msg)
    if content:
        print(f"  [{role}] {content[:150]}")
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"    -> Calling: {tc['name']}({tc['args']})")

Agent conversation:
  [human] What is LCEL? Also calculate 42 * 17.
    -> Calling: search_knowledge({'query': 'LCEL'})
    -> Calling: calculate({'expression': '42 * 17'})
  [tool] LCEL (LangChain Expression Language) uses the pipe operator to compose chain components.
  [tool] 714
  [ai] LCEL (LangChain Expression Language) uses the pipe operator to compose chain components. 

Additionally, the result of \( 42 \times 17 \) is 714.


## 5. Graph with Gemini Model

Let's swap the model to Gemini and see how it handles the same graph.

In [8]:
# Rebuild with Gemini
gemini_with_tools = GEMINI_MODEL.bind_tools(tools)

def gemini_agent_node(state: ChatState):
    return {"messages": [gemini_with_tools.invoke(state["messages"])]}

gemini_graph = StateGraph(ChatState)
gemini_graph.add_node("agent", gemini_agent_node)
gemini_graph.add_node("tools", tool_node)  # Reuse tool_node from above
gemini_graph.add_edge(START, "agent")
gemini_graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
gemini_graph.add_edge("tools", "agent")

compiled_gemini = gemini_graph.compile()

result = compiled_gemini.invoke({
    "messages": [{"role": "user", "content": "Search for info about RAG, then calculate 100 / 4."}]
})

print("Gemini agent conversation:")
for msg in result["messages"]:
    role = msg.type if hasattr(msg, "type") else "unknown"
    content = msg.content if hasattr(msg, "content") else str(msg)
    if content:
        print(f"  [{role}] {content[:150]}")
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"    -> Calling: {tc['name']}({tc['args']})")

Gemini agent conversation:
  [human] Search for info about RAG, then calculate 100 / 4.
    -> Calling: search_knowledge({'query': 'RAG (Retrieval-Augmented Generation)'})
  [tool] RAG combines retrieval with generation to ground LLM responses in factual data.
    -> Calling: calculate({'expression': '100 / 4'})
  [tool] 25.0
  [ai] [{'type': 'text', 'text': '**Retrieval-Augmented Generation (RAG)** is a technique used to improve the accuracy and reliability of large language models (LLMs). It works by combining the generative capabilities of a model with a retrieval system that pulls relevant, up-to-date information from external data sources (like documents or databases). This process "grounds" the model\'s responses in factual data, reducing the likelihood of hallucinations.\n\nThe result of **100 / 4** is **25**.'}]


## 6. Visualizing Your Graph

LangGraph can generate a visual representation of your graph structure.

In [9]:
# Visualize the graph as ASCII
try:
    print(compiled_agent.get_graph().draw_ascii())
except Exception as e:
    print(f"ASCII visualization requires 'grandalf' package: {e}")
    print("\nGraph structure:")
    print("  START -> agent -> (tools | END)")
    print("  tools -> agent (loop back)")

ASCII visualization requires 'grandalf' package: Install grandalf to draw graphs: `pip install grandalf`.

Graph structure:
  START -> agent -> (tools | END)
  tools -> agent (loop back)


## Summary

| Concept | What You Learned |
|---------|------------------|
| RunnableParallel | Run multiple chains simultaneously |
| RunnableLambda | Wrap Python functions as chain steps |
| RunnablePassthrough | Pass-through and enrich input data |
| StateGraph | Define graph-based agent workflows |
| Conditional Edges | Route between nodes based on state |
| Tool Integration | Bind tools to models in a graph |
| Agent Loop | Create cycles for iterative tool use |

**Next:** [03 - Advanced: LangGraph Deep Dive](./03_advanced_langgraph.ipynb)